Importar requisitos prueba
=====================

Cargamos requisitos de prueba en cosmoSys-Req.

Comenzamos conectándonos a la instancia de cosmoSys-Req a donde importaremos los requisitos, y listamos los proyectos activos.

Utilizamos el fichero configfile_req.py donde guardaremos las opciones de conexión de cosmoSys-Req y otras variables de configuración necesarias para que el código funcione.


In [1]:
from cfg.configfile_req import *
from redminelib import Redmine

redmine = Redmine(req_server_url,key=req_key_txt)
projects = redmine.project.all()

print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

    

Proyectos:
	 mytest  	|  MyTest
	 reqtest  	|  ReqTest
	 reqtest3  	|  ReqTest3
	 rqtest4  	|  RqTest4


Ahora vamos a seleccionar sólo un proyecto: aquél que tiene el identificador almacenado en configfile_req.py dentro de la variable 'req_project_id_str':

In [2]:
my_project = redmine.project.get(req_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

Obtenemos proyecto:  mytest  |  MyTest


Vamos a recorrer la hoja de cálculo con los datos de carga especificada en la variable const_file_name del fichero configfile_req.py.  Listaremos los datos cargados.

In [3]:
from pyexcel_ods import get_data

file_path = "." + "/uploading/" + req_upload_file_name

data = get_data(file_path, start_column=req_upload_start_column, column_limit=(req_upload_end_column-req_upload_start_column+1),
                start_row=req_upload_start_row, row_limit=(req_upload_end_row - req_upload_start_row+1))
data

OrderedDict([('Intro',
              [[],
               ['Copy RootDoc tab to as many requirement documents you need in the project'],
               ['In the first row of each tab, fill the document attributes, including the Tab name of the parent'],
               ['Fill the requirements at the table.'],
               ['To set the relationships between requirements, use the RqID'],
               ['The tab Dict is giving some categories needed to automatically create the drop-down lists and the links to the database or the diagrams.'],
               [],
               []]),
             ('Dict',
              [['Req server URL',
                'http://localhost:5555',
                '',
                '',
                '',
                'RqTarget values',
                'RqStatus values',
                '',
                'marker column'],
               ['', '', '', '', '', 'v0.1', 'RqDraft', '', 1],
               ['', '', '', '', '', 'v0.2', 'RqStable', '', 2],
      

Recorreremos ahora las diferentes pestañas para sacar los documentos de requisitos que debemos crear

In [4]:
my_project_versions = []
for v in my_project.versions:
    my_project_versions += [v]

for t in data:
    print("********************"+t)
    #primero filtramos la pestaña Intro
    if (t!='Intro'):
        if (t=='Dict'):
            # Cargamos las versiones
            # TODO
            dataimport = data[t]
            rowindex = req_upload_version_startrow
            while (rowindex <= req_upload_version_endrow):
                d = dataimport[rowindex]
                rowindex += 1
                thisversion = d[req_upload_version_column]
                if (len(thisversion)>0):
                    print(rowindex,": ",thisversion)
                    findVersionSuccess = False
                    thisVersionId = None
                    for v in my_project_versions:
                        if not(findVersionSuccess):
                            if (v.name == thisversion):
                                findVersionSuccess = True
                                print("la versión ",thisversion," ya existe")
                    
                    if not findVersionSuccess:
                        print("la versión ",thisversion," no existe")
                        version = redmine.version.create(
                            project_id = my_project.identifier,
                            name = thisversion,
                            status = 'open',
                            sharing = 'hierarchy',
                            description = thisversion,
                        )
                        my_project_versions += [version]
                        thisVersionId = version.id
                        print("he creado la versión con nombre ",version.name)

        else:
            # Obtenemos el contenido de esa pestaña
            dataimport = data[t]
            # Como identificador del documento y "subject", tomamos el nombre de la pestaña
            docidstr = t
            print("DocID: "+docidstr)
            # Obtenemos la información de la fila donde están los datos adicionales del documento de requisito
            d = dataimport[req_upload_doc_row]
            # Como nombre del documento y "description", tomamos la columna de docname
            docname = d[req_upload_doc_name_column]
            print("DocName: ",docname)
            # Como prefijo de los códigos generados por el documento, tomamos la columna de prefijo
            prefixstr = d[req_upload_doc_prefix_column]
            # Usando el identificador del documento, determinamos si éste ya existe o hay que crearlo
            doclist = redmine.issue.filter(project_id=req_project_id_str, subject=docidstr, tracker_id=req_doc_tracker_id)
            if(len(doclist)==0):
                # no existe el requisito asociado a la pestaña, lo creo
                print ("Creando documento ",docidstr)
                thisreq = redmine.issue.create(project_id = req_project_id_str,
                                               tracker_id = req_doc_tracker_id,
                                               subject = docidstr,
                                               description = docname,
                                               custom_fields=[{'id': req_title_cf_id,'value': docname},
                                                              {'id': req_prefix_cf_id, 'value': prefixstr},
                                                              {'id': req_chapter_cf_id, 'value': prefixstr}
                                                             ]
                                              )
            else:
                # existe el requisito, así que lo actualizo
                print ("actualizando documento", doclist[0].id)
                redmine.issue.update(resource_id=doclist[0].id,
                                     description = docname,
                                     custom_fields=[{'id': req_title_cf_id,'value': docname},
                                                    {'id': req_prefix_cf_id, 'value':prefixstr},
                                                    {'id': req_chapter_cf_id, 'value': prefixstr}
                                                   ]
                                     )


********************Intro
********************Dict
2 :  v0.1
la versión  v0.1  no existe
he creado la versión con nombre  v0.1
3 :  v0.2
la versión  v0.2  no existe
he creado la versión con nombre  v0.2
4 :  v0.3
la versión  v0.3  no existe
he creado la versión con nombre  v0.3
5 :  v1.0
la versión  v1.0  no existe
he creado la versión con nombre  v1.0
********************SR
DocID: SR
DocName:  Parent system requirements
Creando documento  SR
********************SRC
DocID: SRC
DocName:  Child system requirements
Creando documento  SRC


Vamos a buscar los documentos padre de los documentos ya existentes

In [5]:
for t in data:
    print("********************"+t)
    # No vamos a tratar la pestaña "Intro"
    if (t!='Intro') and (t!='Dict'):
        # importamos los datos 
        dataimport = data[t]
        # Buscamos el documento que representa a la pestaña
        doclist = redmine.issue.filter(project_id=req_project_id_str, subject=t, tracker_id=req_doc_tracker_id)
        if(len(doclist)>0):
            # Existe el documento asociado a la pestaña, puedo tratarlo
            thisdoc = doclist[0]
            print("rqid: "+thisdoc.subject)
            d = dataimport[0]
            parentdocstr = d[req_upload_doc_parent_column]
            print("parent str: " + parentdocstr)
            if (len(parentdocstr)>0):
                # Se ha especificado un documento padre
                parentdoclist = redmine.issue.filter(project_id=req_project_id_str,subject=parentdocstr, tracker_id=req_doc_tracker_id)
                if len(parentdoclist)>0:
                    # Existe el documento padre, puedo tratarlo
                    parentdoc = parentdoclist[0]
                    print("parent: ",parentdoc)
                    print("parent id:",parentdoc.id)
                    redmine.issue.update(resource_id=thisdoc.id, parent_issue_id = parentdoc.id)
                else:
                    print("error, no encontramos el padre")


********************Intro
********************Dict
********************SR
rqid: SR
parent str: 
********************SRC
rqid: SRC
parent str: SR
parent:  SR
parent id: 67


Ya tenemos los documentos de requisitos (pestañas) cargados como requisitos padres.  Ahora puedo recorrer los requisitos de cada una de las pestañas y crear los requisitos que falten.

In [6]:
for t in data:
    print("********************"+t)
    if (t!='Intro') and (t!='Dict'):
        dataimport = data[t]
        reqdoclist = redmine.issue.filter(project_id=req_project_id_str, status_id='*',subject=t)
        print(t, "len list: ",len(reqdoclist))
        if(len(reqdoclist)>0):
            reqDoc = reqdoclist[0]
            print("reqDocId:", reqDoc.id)
            reqDocPrefix = reqDoc.custom_fields.get(req_prefix_cf_id).value
            print("reqDocPrefix:",reqDocPrefix)
            # Obtendremos en un bucle todos los requisitos, saltándonos las 2 líneas iniciales
            current_row = 0
            for r in dataimport:
                if current_row < req_upload_first_row:
                    current_row += 1
                else:
                    title_str = r[req_upload_title_column]
                    print("title: ",title_str)
                    if len(title_str) <= 0:
                        print("Saltamos una fila que no tiene suficientes celdas")
                    else:
                        # Estamos procesando las líneas de requisitos
                        rqidstr = r[req_upload_id_column]
                        print("rqid: "+rqidstr)
                        related_str = r[req_upload_related_column]
                        print("related: "+related_str)
                        title_str = r[req_upload_title_column]
                        print("title: ",title_str)
                        descr = r[req_upload_descr_column]
                        print("description: ",descr)
                        reqsource = r[req_upload_source_column]
                        print("reqsource: ",reqsource)
                        reqtype = r[req_upload_type_column]
                        print("reqtype: ",reqtype)
                        reqlevel = r[req_upload_level_column]
                        print("reqlevel: ",reqlevel)
                        reqrationale = r[req_upload_rationale_column]
                        print("reqrationale: ",reqrationale)
                        reqvar = r[req_upload_var_column]
                        print("reqvar: ",reqvar)
                        reqvalue = r[req_upload_value_column]
                        print("reqvalue: ",reqvalue)
                        rqchapter = reqDocPrefix+str(r[req_upload_chapter_column])
                        print("rqchapter: ",rqchapter)
                        rqtarget = r[req_upload_target_column]
                        print("rqtarget: ",rqtarget)
                        findVersionSuccess = False
                        thisVersionId = None
                        print("num versiones: ",len(my_project_versions))
                        for v in my_project_versions:
                            print("version: ",v)
                            if not(findVersionSuccess):
                                print("--->",v.name,":",rqtarget)
                                if (v.name == rqtarget):
                                    print("LO ENCONTRÉ!!")
                                    findVersionSuccess = True
                                    thisVersionId = v.id
                                else:
                                    print("NO.....")
                                
                            else:
                                print("this version succes????:",findVersionSuccess)

                        print("thisVersionId: ",thisVersionId)
                        # A partir del nombre del requisito, miramos si el requisito ya existe
                        reqlist = redmine.issue.filter(project_id=req_project_id_str,subject=rqidstr, tracker_id=req_rq_tracker_id)
                        if(len(reqlist)==0):
                            # no existe el requisito, lo creo
                            print ("creando requisito")
                            thisreq = redmine.issue.create(project_id = req_project_id_str,
                                                           tracker_id = req_rq_tracker_id,
                                                           subject = rqidstr,
                                                           description = descr,
                                                           fixed_version_id = thisVersionId,
                                                           custom_fields=[{'id': req_title_cf_id,'value': title_str},
                                                                          {'id': req_sources_cf_id,'value': reqsource},
                                                                          {'id': req_type_cf_id,'value': reqtype},
                                                                          {'id': req_level_cf_id,'value': reqlevel},
                                                                          {'id': req_rationale_cf_id,'value': reqrationale},
                                                                          {'id': req_var_cf_id,'value': reqvar},
                                                                          {'id': req_value_cf_id,'value': reqvalue},
                                                                          {'id': req_chapter_cf_id, 'value':rqchapter}
                                                                         ]
                                                          )


                        else:
                            # existe el requisito, así que lo actualizo
                            print ("actualizando requisito", reqlist[0].id)
                            redmine.issue.update(resource_id=reqlist[0].id,
                                                 description = descr,
                                                 fixed_version_id = thisVersionId,
                                                 custom_fields=[{'id': req_title_cf_id,'value': title_str},
                                                                {'id': req_sources_cf_id,'value': reqsource},
                                                                {'id': req_type_cf_id,'value': reqtype},
                                                                {'id': req_level_cf_id,'value': reqlevel},
                                                                {'id': req_rationale_cf_id,'value': reqrationale},
                                                                {'id': req_var_cf_id,'value': reqvar},
                                                                {'id': req_value_cf_id,'value': reqvalue},
                                                                {'id': req_chapter_cf_id, 'value':rqchapter}
                                                                ]
                                                )




********************Intro
********************Dict
********************SR
SR len list:  1
reqDocId: 67
reqDocPrefix: SR-
title:  Introduction
rqid: SR-0001
related: 
title:  Introduction
description:  
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  This document is the root document for the requirements of the system SYS
reqvar:  
reqvalue:  
rqchapter:  SR-1.
rqtarget:  v0.1
num versiones:  4
version:  v0.1
---> v0.1 : v0.1
LO ENCONTRÉ!!
version:  v0.2
this version succes????: True
version:  v0.3
this version succes????: True
version:  v1.0
this version succes????: True
thisVersionId:  13
creando requisito
title:  About this document
rqid: SR-0002
related: 
title:  About this document
description:  The SYS system has to fulfill the requirements in this document.
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  The document needs a readable structure
reqvar:  
reqvalue:  
rqchapter:  SR-1.1
rqtarget:  v0.1
num versiones:  4
version:  v0.1
---> v0.1 : v0.1
LO E

title:  About this document
rqid: SRC-0002
related: 
title:  About this document
description:  The CHILD system has to fulfill the requirements in this document.
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  The document needs a readable structure
reqvar:  
reqvalue:  
rqchapter:  SRC-1.1
rqtarget:  v0.1
num versiones:  4
version:  v0.1
---> v0.1 : v0.1
LO ENCONTRÉ!!
version:  v0.2
this version succes????: True
version:  v0.3
this version succes????: True
version:  v1.0
this version succes????: True
thisVersionId:  13
creando requisito
title:  Requirements
rqid: SRC-0003
related: 
title:  Requirements
description:  
reqsource:  
reqtype:  Info
reqlevel:  System
reqrationale:  This section includes the documents of the CHILD system.  It is divided into two main sections: functional requirements and non-functional requirements.
reqvar:  
reqvalue:  
rqchapter:  SRC-2.
rqtarget:  v0.1
num versiones:  4
version:  v0.1
---> v0.1 : v0.1
LO ENCONTRÉ!!
version:  v0.2
this versio

Ahora buscamos las relaciones entre requisitos padres e hijos, y de dependencia.

In [7]:
for t in data:
    print("********************"+t)
    if (t!='Intro') and (t!='Dict'):
        dataimport = data[t]
        reqdoclist = redmine.issue.filter(project_id=req_project_id_str, status_id='*',subject=t, tracker_id=req_doc_tracker_id)
        print(t, "len list: ",len(reqdoclist))
        if(len(reqdoclist)>0):
            reqDoc = reqdoclist[0]
            print("reqDocId:", reqDoc.id)
            # Obtendremos en un bucle todos los requisitos, saltándonos las 2 líneas iniciales
            current_row = 0
            for r in dataimport:
                if current_row < req_upload_first_row:
                    current_row += 1
                else:
                    title_str = r[req_upload_title_column]
                    if len(title_str) <= 0:
                        print("Saltamos una fila que no tiene suficientes celdas")
                    else:
                        # Estamos procesando las líneas de requisitos
                        rqidstr = r[req_upload_id_column]
                        print("rqid: "+rqidstr)
                        related_str = r[req_upload_related_column]
                        print("related: "+related_str)
                        parent_str = r[req_upload_parent_column]
                        print("parent_str: ",parent_str)
                        # Accedo al objeto requisito
                        reqlist = redmine.issue.filter(project_id=req_project_id_str,subject=rqidstr, tracker_id=req_rq_tracker_id)
                        if(len(reqlist)>0):
                            # El requisito existe, lo tomo como requisito actual
                            current_req = reqlist[0];
                            if (len(parent_str)>0):
                                # Buscamos el requisito padre para enlazarlo
                                parentlist = redmine.issue.filter(project_id=req_project_id_str,subject=parent_str)
                                if len(parentlist)>0:
                                    # Existe el documento padre, puedo tratarlo
                                    parentreq = parentlist[0]
                                    print("parent: ",parentreq)
                                    print("parent id:",parentreq.id)
                                    redmine.issue.update(resource_id=current_req.id, parent_issue_id = parentreq.id)                                    
                                else:
                                    print("ERROR: No encontramos el requisito padre!!!")

                            else:
                                # El requisito no tiene padre, así que su padre será el documento
                                redmine.issue.update(resource_id=current_req.id, parent_issue_id = reqDoc.id)                                    
                                
                            # Exploramos ahora las relaciones de dependencia
                            if (len(related_str)>0):
                                if (related_str[0]!='-'):
                                    # Busco las relaciones existences con este requisito
                                    my_req_relations = redmine.issue_relation.filter(issue_id=current_req.id)
                                    # Como voy a tratar las que tienen el requisito como destino, las filtro
                                    my_filtered_req_relations = list(filter(lambda x: x.issue_to_id == current_req.id, my_req_relations))
                                    # Ahora saco todos los ID de los requisitos del otro lado (en el lado origen de la relación)
                                    related_req = related_str.split()
                                    # Y voy a recorrer uno a uno
                                    for rreq in related_req:
                                        print("related to: ",rreq)
                                        # Busco ese requisito
                                        blocking_reqlist = redmine.issue.filter(project_id=req_project_id_str,subject=rreq, tracker_id=req_rq_tracker_id)
                                        if (len(blocking_reqlist)>0):
                                            blocking_req = blocking_reqlist[0]
                                            # Veo si ya existe algún tipo de relación con él
                                            preexistent_relations = list(filter(lambda x: x.issue_id == blocking_req.id, my_filtered_req_relations))
                                            print(preexistent_relations)
                                            if (len(preexistent_relations)>0):
                                                print("Ya existe la relación")
                                            else:
                                                print("Creo una nueva relación")
                                                relation = redmine.issue_relation.create(
                                                    issue_id=blocking_req.id,
                                                    issue_to_id=current_req.id,
                                                    relation_type='blocks'
                                                )


********************Intro
********************Dict
********************SR
SR len list:  1
reqDocId: 67
rqid: SR-0001
related: 
parent_str:  
rqid: SR-0002
related: 
parent_str:  SR-0001
parent:  SR-0001
parent id: 69
rqid: SR-0012
related: 
parent_str:  SR-0001
parent:  SR-0001
parent id: 69
rqid: SR-0003
related: 
parent_str:  
rqid: SR-0004
related: 
parent_str:  SR-0003
parent:  SR-0003
parent id: 72
rqid: SR-0005
related: 
parent_str:  SR-0004
parent:  SR-0004
parent id: 73
rqid: SR-0006
related: 
parent_str:  SR-0004
parent:  SR-0004
parent id: 73
rqid: SR-0007
related: SR-0005 SR-0006
parent_str:  SR-0004
parent:  SR-0004
parent id: 73
related to:  SR-0005
[]
Creo una nueva relación
related to:  SR-0006
[]
Creo una nueva relación
rqid: SR-0008
related: SR-0005
parent_str:  SR-0004
parent:  SR-0004
parent id: 73
related to:  SR-0005
[]
Creo una nueva relación
rqid: SR-0009
related: 
parent_str:  SR-0003
parent:  SR-0003
parent id: 72
rqid: SR-0010
related: 
parent_str:  SR-0009
pa

rqid: SRC-0002
related: 
parent_str:  SRC-0001
parent:  SRC-0001
parent id: 81
rqid: SRC-0003
related: 
parent_str:  
rqid: SRC-0004
related: 
parent_str:  SRC-0003
parent:  SRC-0003
parent id: 83
rqid: SRC-0005
related: SR-0004
parent_str:  SRC-0004
parent:  SRC-0004
parent id: 84
related to:  SR-0004
[]
Creo una nueva relación
rqid: SRC-0006
related: 
parent_str:  SRC-0004
parent:  SRC-0004
parent id: 84
rqid: SRC-0007
related: SRC-0005 SRC-0006
parent_str:  SRC-0004
parent:  SRC-0004
parent id: 84
related to:  SRC-0005
[]
Creo una nueva relación
related to:  SRC-0006
[]
Creo una nueva relación
rqid: SRC-0008
related: SRC-0005
parent_str:  SRC-0004
parent:  SRC-0004
parent id: 84
related to:  SRC-0005
[]
Creo una nueva relación
rqid: SRC-0009
related: 
parent_str:  SRC-0003
parent:  SRC-0003
parent id: 83
rqid: SRC-0010
related: 
parent_str:  SRC-0009
parent:  SRC-0009
parent id: 89
Saltamos una fila que no tiene suficientes celdas
Saltamos una fila que no tiene suficientes celdas
Sa